In [1]:
from Utils.preprocessing import *
from Utils.pipelineComponents import *
from Utils.customPipelines import *

RANDOM_STATE = 2024

# Webscraping Portion

In [2]:
# Read prof's sample data to view format & data science roles
sample_data_a = pd.read_csv("Data/jobs_in_data.csv")
sample_data_b = pd.read_csv("Data/QF632_Project_2_b.csv")
sample_data_c = pd.read_csv("Data/QF632_Project_2_c.csv")
sample_data_d = pd.read_csv("Data/Glassdoor_Salary_Cleaned_Version.csv")

In [3]:
base_url = "https://h1bdata.info/index.php?em=&job=data+scientist&city=&year=all+years"
roles = ["data scientists", "data analyst", "data storytellers", "data insights", "research scientist", "business data analyst",
        "data analytics", "data engineer", "machine learning", "ml engineer", "ai architect"]

all_df = pd.DataFrame()
for role in roles:
    job = "+".join(role.split(" "))
    url = base_url + "/index.php?em=&job=" + job + "&city=&year=all+years"
    dfs_base = pd.read_html(url)
    
    # to include all other variations, add a '&'
    url = base_url + "/index.php?em=&job=" + job + "+&&city=&year=all+years"
    dfs_var = pd.read_html(url)
    
    temp_df = pd.concat([dfs_base[0], dfs_var[0]])
    temp_df["TYPE"] = role
    
    all_df = pd.concat([all_df, temp_df])

print(all_df.shape)
all_df.sample(5)

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,TYPE,Unnamed: 6
883,SQUARETRADE INC,BUSINESS DATA ANALYST,95000,"SAN FRANCISCO, CA",12/02/2021,12/20/2021,business data analyst,NaN
1706,USAA FEDERAL SAVINGS BANK,DATA ENGINEER I,125000,"SAN ANTONIO, TX",02/23/2023,03/20/2023,data engineer,NaN
5274,NYU GROSSMAN SCHOOL OF MEDICINE,RESEARCH SCIENTIST,90000,"NEW YORK, NY",09/20/2023,12/01/2023,research scientist,NaN
5889,IKCON TECHNOLOGIES INC,DATA ANALYST,74589,"JACKSONVILLE, FL",09/09/2021,09/17/2021,data analyst,NaN
11776,ANKR PBC,DATA ANALYST,122000,"SAN FRANCISCO, CA",05/11/2020,10/01/2020,data analyst,NaN


# Cleaning Data

In [24]:
all_df["START DATE"] = pd.to_datetime(all_df["START DATE"], errors='coerce')
all_df["SUBMIT DATE"] = pd.to_datetime(all_df["SUBMIT DATE"], errors='coerce')
all_df = all_df.drop_duplicates()

all_df2 = all_df.dropna(subset = ["BASE SALARY"]).copy()
all_df2 = all_df2.iloc[:, :-1].reset_index(drop=True)
all_df2["YEAR"] = pd.DatetimeIndex(all_df2['START DATE']).year

print(all_df2.shape)
all_df2.sample(5)

(52252, 8)


,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,TYPE,YEAR
31187,ITECH-GO LLC,DATA ENGINEER,75000,"NOVI, MI",2022-01-21,2022-03-15,data engineer,2022.0
26575,SEATTLE CHILDREN'S HOSPITAL,RESEARCH SCIENTIST III,71351,"SEATTLE, WA",2022-06-22,2022-08-19,research scientist,2022.0
27193,HOLISTIC PARTNERS INC,BUSINESS DATA ANALYST,76000,"MIAMI, FL",2019-03-15,2019-09-02,business data analyst,2019.0
31052,LORVISH TECHNOLOGIES INC,DATA ENGINEER,74443,"APEX, NC",2023-06-20,2023-10-01,data engineer,2023.0
1068,COLLABORAIT INC,DATA ANALYST,94000,"HERNDON, VA",2020-08-21,2020-09-15,data analyst,2020.0
